# Анализ тональности

Анализ тональности текста (англ. Sentiment analysis) — задача компьютерной лингвистики, заключающаяся в определении эмоциональной окраски (тональности) текста и, в частности, в выявлении эмоциональной оценки авторов по отношению к объектам, описываемым в тексте.

Мы не будем углубляться в то, как инструменты, с которыми мы работаем, устроены внутри, а возьмём готовые решения и будем работать с результатами.

[Эта тетрадка на Google Colab](https://colab.research.google.com/drive/1aDb4yPa4hKLVJOzR3k_oDIgheaOIAh32?usp=sharing).

In [1]:
%%capture
#Установка библиотеки transformers (если еще не установлена)
!pip install transformers

In [9]:
import pandas as pd
import requests
from transformers import pipeline

In [2]:
#Загрузка предобученной модели для русского языка
sentiment_model = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


### Модели для других релевантных языков (не всегда та же библиотека)

- английский: [на данных Твиттера](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment), [на более широких данных](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) и многие другие
- японский: [один](https://github.com/Hironsan/asari), [два](https://huggingface.co/jarvisx17/japanese-sentiment-analysis)
- китайский: [один](https://medium.com/analytics-vidhya/python-snownlp-sentiment-analysis-for-the-chinese-language-8d9cafd0447d)

Частности в использовании моделей могут отличаться — смотрите описывающие их страницы или обращайтесь лично за помощью.

In [4]:
#Библиотека выдаёт значение для текста, который ей подашь
#Число, которое даёт степень уверенности и вердикт

sentiment_model('Отвратительный сервис, никогда больше не приду!!!')

[{'label': 'NEGATIVE', 'score': 0.7515146732330322}]

In [5]:
sentiment_model('Самый вкусный шоколад в моей жизни!')

[{'label': 'POSITIVE', 'score': 0.8740137815475464}]

In [6]:
sentiment_model('В Подмосковье были ограблены три квартиры.')

[{'label': 'NEUTRAL', 'score': 0.8062561750411987}]

In [8]:
#Получаем сам вердикт

sentiment_model('Самый вкусный шоколад в моей жизни!')[0]['label']

'POSITIVE'

Применим инструмент к набору данных, состоящих из Твитов. Проверим, правда ли, что люди с большим количеством друзей чаще пишут позитивные вещи.

In [10]:
#Скачиваем тексты
data = pd.read_csv('https://raw.githubusercontent.com/alekseyst/text_analysis_2025/refs/heads/main/Practical_8/twits.csv')

In [11]:
#Определяем тональность для каждого текста в таблице
data['predicted_label'] = data['text'].apply(lambda x: sentiment_model(x)[0]['label'])
data

,text,n_friends,predicted_label
0,"@first_timee хоть я и школота, но поверь, у на...",61,POSITIVE
1,"Да, все-таки он немного похож на него. Но мой ...",31,POSITIVE
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,27,POSITIVE
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",17,POSITIVE
4,@irina_dyshkant Вот что значит страшилка :D\nН...,23,POSITIVE
...,...,...,...
462,"Тошнит, голова болит, состояние дрянь, работ...",74,NEGATIVE
463,@umnitsa_julia ещё и ребёнка вперёд посадила.....,43,NEGATIVE
464,@yufins совсем тви забросил.Не пишет ничего:(,63,NEGATIVE
465,Очень было стыдно перед классным руководителем!(,19,NEGATIVE


Готово! Мы приписали тональность для каждого текста! Теперь можем использовать её для любых нужных нам целей — например, посмотреть, как это значение связано с количеством друзей.

In [20]:
#Установим 100 друзей как граничное значение и аннотируем в соответствии с ним тексты 
data['many_friends'] = data['n_friends'].apply(lambda x: 1 if (x >= 100) else 0)
data

,text,n_friends,predicted_label,many_friends
0,"@first_timee хоть я и школота, но поверь, у на...",61,POSITIVE,0
1,"Да, все-таки он немного похож на него. Но мой ...",31,POSITIVE,0
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,27,POSITIVE,0
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",17,POSITIVE,0
4,@irina_dyshkant Вот что значит страшилка :D\nН...,23,POSITIVE,0
...,...,...,...,...
462,"Тошнит, голова болит, состояние дрянь, работ...",74,NEGATIVE,0
463,@umnitsa_julia ещё и ребёнка вперёд посадила.....,43,NEGATIVE,0
464,@yufins совсем тви забросил.Не пишет ничего:(,63,NEGATIVE,0
465,Очень было стыдно перед классным руководителем!(,19,NEGATIVE,0


In [29]:
#Построим таблицу сопряжённости
cont_table = pd.crosstab(data['many_friends'], data['predicted_label'])
cont_table

predicted_label,NEGATIVE,POSITIVE
many_friends,,
0,185,133
1,75,74


In [30]:
#Её далее можно использовать лдля статистических расчётов
from scipy import stats
stats.chi2_contingency(cont_table)

Chi2ContingencyResult(statistic=np.float64(2.219691391598547), pvalue=np.float64(0.136260294366453), dof=1, expected_freq=array([[177.04496788, 140.95503212],
       [ 82.95503212,  66.04496788]]))